In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn import metrics

In [2]:
data = pd.read_csv('../04_merged_content/dataset.csv', sep = ';',
                   names=['name','cod','words'], index_col='name', header=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 570 entries, 02001000312017_6699679_20160112_MANDADO_DE_CITACAO.pdf to 02000012462017_8103027_peticao_inicial_0000803-08.2017.5.11.0017.pdf
Data columns (total 2 columns):
cod      570 non-null int64
words    569 non-null object
dtypes: int64(1), object(1)
memory usage: 13.4+ KB


In [3]:
na = data[data.words.isnull()].index.values
data[data.words.isnull()]

,cod,words
name,,
02000008202017_7611302_20170413_FEDERAL_SEGUROS_-_00820_2017_-_0224726-83.2011.8.04.0001_16.pdf,5083,NaN


In [4]:
data.drop(na, inplace=True)
data.shape

(569, 2)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data.words, data.cod, test_size = 0.2, random_state=42)
print(len(x_train))
print(len(x_test))

455
114


In [6]:
vec = CountVectorizer()
vec_train = vec.fit_transform(x_train)
vec_train.shape

(455, 39562)

In [7]:
clf = MultinomialNB().fit(vec_train, y_train)

In [8]:
vec_test = vec.transform(x_test)
predictions = clf.predict(vec_test)
clf.score(vec_test, y_test)

0.956140350877193

In [9]:
print(metrics.classification_report(y_test, predictions))
print(metrics.confusion_matrix(y_test, predictions))

             precision    recall  f1-score   support

       5070       0.95      1.00      0.98       101
       5083       1.00      0.62      0.76        13

avg / total       0.96      0.96      0.95       114

[[101   0]
 [  5   8]]


In [10]:
cls = KMeans(n_clusters=3, random_state=42).fit(vec_train)